# Guider mode quick look
This is intended to unpack the rawStamps from guider mode FITS files and 
give a quick look of all 16 stamps for all 4 CCDs. \
Craig Lage - 03-Dec-24

In [ ]:
import os
import shlex, subprocess
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import numpy as np
from astropy.io import fits
from lsst.resources import ResourcePath

# Get the main header and the information it contains

In [ ]:
def getMainHeaderInfo(hdu_list):
    hdr0 = hdu_list[0].header
    #raft = hdr0['RAFTBAY']
    #ccd = hdr0['CCDSLOT']
    #dayObs = hdr0['DAYOBS']
    #seqNum = hdr0['SEQNUM']
    roiCols = hdr0['ROICOLS']
    roiRows = hdr0['ROIROWS']
    try:
        roiUnder = hdr0['ROIUNDER']
    except:
        roiUnder = 6
    nStamps = hdr0['N_STAMPS']
    
    # Set the xor value - Guider CCDs are different from science CCDs
    if raft in ['R00', 'R04', 'R40', 'R44']:
        # Guider rafts
        xor = 0x20000
    else:
        # Science rafts
        xor = 0x1ffff
    return [roiRows, roiCols, roiUnder, nStamps, xor]

# Now define the code to unpack the rawStamps:

In [ ]:
def unpackStamps(hduNum):
    data = (hdu_list[hduNum].data['rawStamp'][0]).astype('>u4')
    data.byteswap(inplace=True)
    totalCols = roiCols + roiUnder
    size = roiRows * totalCols
    out = np.zeros([16, size], dtype=int)
    image_out = np.zeros([16, roiRows, roiCols], dtype=int)
    
    for n in range(size):
        # Get 9 32 bit words of data
        res = ''
        for i in range(9):
            d = data[(size - n) * 9 - i - 1]
            d = format(d, '#034b')
            d = d.split('b')[1]
            res += d
        # Now extract 16 18 bit words from the data
        for i in range(16):
            bin_value = res[i * 18:(i + 1) * 18]
            int_value = int(bin_value, 2)
            final_value = int_value ^ xor
            out[i,n] = final_value  
    for i in range(16):
        reshaped = out[i,:].reshape(roiRows, totalCols)
        image_out[i,:,:] = np.flipud(np.fliplr(reshaped[:,0:roiCols]))
    return image_out

In [ ]:
# Set the scaling
autoscale = True
# Scale to use if autoscale = False
vmin = 10000
vmax = 30000

# Get a quick look at one frame for all 4 CCDs

In [ ]:
%matplotlib inline

dayObs = 20241204
seqNum = 134
expId = int(f"{dayObs}{seqNum:05d}")
raft = 'R22'
ccds = ['S00', 'S01', 'S21', 'S22']
n = 2 # This just unpacks one frame.  This chooses which frame

for ccd in ccds:
    filename = f"s3://embargo@rubin-summit/LSSTComCam/{dayObs}/CC_O_{dayObs}_{seqNum:06d}/CC_O_{dayObs}_{seqNum:06d}_{raft}_{ccd}_guider.fits"
    
    rp = ResourcePath(filename)
    with rp.open(mode="rb") as f:
        hdu_list = fits.open(f)
    [roiRows, roiCols, roiUnder, nStamps, xor] = getMainHeaderInfo(hdu_list)
    hduNum = 2 * n + 1
    hdrn = hdu_list[hduNum].header
    timestamp = hdrn['STMPTIME']
    image_out = unpackStamps(hduNum)
    fig = plt.figure(figsize=(10,10))
    axs = fig.subplots(4,4)
    plt.subplots_adjust(wspace=.7, hspace=-0.2)
    plt.suptitle(f"Guider mode {raft} {ccd} {expId}, Frame {n+1}\n{timestamp}", fontsize=24)
    for i in range(4):
        for j in range(4):
            seg = i + 4 * j
            axs[i][j].set_title(f"segment {seg}")
            if autoscale:
                im = axs[i][j].imshow(image_out[seg], interpolation='nearest', origin='upper', cmap='Greys')
            else:
                im = axs[i][j].imshow(image_out[seg], interpolation='nearest', origin='upper', vmin=vmin, vmax=vmax, cmap='Greys')
            divider = make_axes_locatable(axs[i][j])
            cax = divider.append_axes("right", size="5%", pad=0.05)
            fig.colorbar(im, cax=cax)
            
    plt.savefig(f"/home/c/cslage/u/Guider_Mode/quick_looks/Guider_QuickLook_{expId}_{raft}_{ccd}.png")

# Quick look at one of the headers

In [ ]:
hduNum = 7
hdr_test = hdu_list[hduNum].header
for key in hdr_test.keys():
    print(key, hdr_test[key])

In [ ]:
seg = 13
arr = np.array(image_out[seg])
arr = arr - np.median(arr)
(ycen, xcen) = np.unravel_index(arr.argmax(), arr.shape)
stamp = 30
x1 = int(xcen - stamp / 2)
x2 = x1 + stamp

y1 = int(ycen - stamp / 2)
y2 = y1 + stamp
print(x1, x2, y1, y2, xcen, ycen)
plt.imshow(arr[y1:y2, x1:x2], interpolation='nearest', origin='lower')#, cmap='Greys')

In [ ]:
np.unravel_index(arr.argmax(), arr.shape)

In [ ]:
arr

In [ ]:
com(arr)